# Recurrent Neural Networks (RNNs), LSTMs & GRUs

## Week 1 - Deep Learning Foundations for Generative AI

In this notebook, we will explore:
1. **Why RNNs?** - Understanding sequence modeling
2. **Basic RNN** - Architecture and limitations
3. **LSTM** - Long Short-Term Memory networks
4. **GRU** - Gated Recurrent Units
5. **Hands-on**: Build a text generation model

---

## 1. Setup & Installation

First, let's install and import the required libraries.

In [ ]:
# Install required packages (uncomment if running on Colab)
# !pip install tensorflow numpy matplotlib

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

---
## 2. Why Recurrent Neural Networks?

### The Problem with Traditional Neural Networks

Traditional feedforward neural networks have a major limitation: **they can't handle sequential data** where the order matters.

Consider these examples:
- **Text**: "The cat sat on the mat" - word order matters!
- **Speech**: Audio signals are time-series
- **Stock prices**: Today's price depends on yesterday's

### The RNN Solution

RNNs introduce the concept of **memory** through a hidden state that gets updated at each time step.

```
     ┌───────────────────────────────────────┐
     │              Hidden State             │
     │         (Memory of the past)          │
     └───────────────────────────────────────┘
              ↑           ↑           ↑
          ┌───┴───┐   ┌───┴───┐   ┌───┴───┐
          │  RNN  │ → │  RNN  │ → │  RNN  │
          └───┬───┘   └───┬───┘   └───┬───┘
              ↑           ↑           ↑
            Input₁      Input₂      Input₃
           ("The")     ("cat")     ("sat")
```

### Mathematical Formulation

At each time step $t$, an RNN computes:

$$h_t = \tanh(W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1} + b_h)$$
$$y_t = W_{hy} \cdot h_t + b_y$$

Where:
- $x_t$ = input at time $t$
- $h_t$ = hidden state at time $t$ (the "memory")
- $y_t$ = output at time $t$
- $W$ = weight matrices
- $b$ = bias vectors

In [ ]:
# Let's build a simple RNN from scratch to understand the concept

class SimpleRNN:
    """A simple RNN implementation for educational purposes."""
    
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights randomly
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Why = np.random.randn(output_size, hidden_size) * 0.01
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
        
    def forward(self, inputs, h_prev):
        """Forward pass through one time step."""
        # Hidden state update
        h_next = np.tanh(self.Wxh @ inputs + self.Whh @ h_prev + self.bh)
        # Output
        y = self.Why @ h_next + self.by
        return y, h_next

# Demo: Process a sequence
rnn = SimpleRNN(input_size=10, hidden_size=20, output_size=5)
h = np.zeros((20, 1))  # Initial hidden state

# Simulate 3 time steps
for t in range(3):
    x = np.random.randn(10, 1)  # Random input
    y, h = rnn.forward(x, h)
    print(f"Time step {t+1}: Output shape = {y.shape}, Hidden state shape = {h.shape}")

---
## 3. The Vanishing Gradient Problem

### Why Basic RNNs Struggle with Long Sequences

When training RNNs with backpropagation through time (BPTT), gradients can:
- **Vanish**: Get smaller and smaller, making long-term learning impossible
- **Explode**: Get larger and larger, causing training instability

This is why simple RNNs can't learn long-term dependencies effectively.

```
Example: Predicting the blank
"I grew up in France... I speak fluent ___"

The word "France" appears many steps before the blank.
Simple RNNs struggle to connect them!
```

In [ ]:
# Demonstration of vanishing gradients

def simulate_gradient_flow(sequence_length, gradient_factor=0.9):
    """Simulate how gradients diminish over time."""
    gradients = [1.0]  # Initial gradient
    for _ in range(sequence_length - 1):
        gradients.append(gradients[-1] * gradient_factor)
    return gradients

# Visualize gradient flow
seq_lengths = [10, 50, 100]
plt.figure(figsize=(12, 4))

for seq_len in seq_lengths:
    grads = simulate_gradient_flow(seq_len)
    plt.plot(grads, label=f'Sequence Length = {seq_len}')

plt.xlabel('Time Steps Back')
plt.ylabel('Gradient Magnitude')
plt.title('Vanishing Gradient Problem in RNNs')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nAfter 100 steps, gradient is only {simulate_gradient_flow(100)[-1]:.10f} of original!")

---
## 4. LSTM - Long Short-Term Memory

LSTMs solve the vanishing gradient problem by introducing **gates** that control information flow.

### LSTM Architecture

```
┌─────────────────────────────────────────────────────────┐
│                      LSTM Cell                          │
│                                                         │
│   ┌─────┐     ┌─────┐     ┌─────┐     ┌─────┐          │
│   │  σ  │     │  σ  │     │ tanh│     │  σ  │          │
│   └──┬──┘     └──┬──┘     └──┬──┘     └──┬──┘          │
│      │           │           │           │              │
│   Forget       Input       Cell       Output           │
│    Gate         Gate      Update       Gate            │
│      │           │           │           │              │
│      ▼           ▼           ▼           ▼              │
│   ╔═════╗     ╔═════╗     ╔═════╗                      │
│   ║  ×  ║ +   ║  ×  ║  →  ║  ×  ║ → hidden state      │
│   ╚═════╝     ╚═════╝     ╚═════╝                      │
│      ↑                                                  │
│   Cell State (Long-term memory highway)                │
└─────────────────────────────────────────────────────────┘
```

### The Three Gates:

1. **Forget Gate**: Decides what to forget from cell state
2. **Input Gate**: Decides what new information to store
3. **Output Gate**: Decides what to output

In [ ]:
# Building an LSTM with Keras

# Create a simple LSTM model
lstm_model = keras.Sequential([
    layers.LSTM(64, input_shape=(10, 32), return_sequences=True),  # 10 time steps, 32 features
    layers.LSTM(32),  # Second LSTM layer
    layers.Dense(10, activation='softmax')  # Output layer
])

lstm_model.summary()

---
## 5. GRU - Gated Recurrent Unit

GRU is a simplified version of LSTM with fewer parameters:

### Key Differences from LSTM:

| Feature | LSTM | GRU |
|---------|------|-----|
| Gates | 3 (forget, input, output) | 2 (reset, update) |
| Cell State | Separate | Combined with hidden state |
| Parameters | More | Fewer |
| Training | Slower | Faster |

### When to use which?
- **LSTM**: When you need to model very long sequences
- **GRU**: When you want faster training with similar performance

In [ ]:
# Building a GRU with Keras

gru_model = keras.Sequential([
    layers.GRU(64, input_shape=(10, 32), return_sequences=True),
    layers.GRU(32),
    layers.Dense(10, activation='softmax')
])

gru_model.summary()

# Compare parameter counts
print(f"\n📊 Parameter Comparison:")
print(f"LSTM Model: {lstm_model.count_params():,} parameters")
print(f"GRU Model:  {gru_model.count_params():,} parameters")
print(f"GRU has {(1 - gru_model.count_params()/lstm_model.count_params())*100:.1f}% fewer parameters!")

---
## 6. 🎯 Hands-On Project: Character-Level Text Generation

Now let's build a complete text generation model using LSTM!

**Goal**: Train a model to generate Shakespeare-like text character by character.

### Steps:
1. Load and preprocess text data
2. Create training sequences
3. Build the LSTM model
4. Train the model
5. Generate new text!

In [ ]:
# Step 1: Load the Shakespeare dataset

# Download Shakespeare text
path_to_file = keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)

# Read the text
text = open(path_to_file, 'r', encoding='utf-8').read()

# Print some statistics
print(f"📚 Dataset Statistics:")
print(f"Total characters: {len(text):,}")
print(f"Unique characters: {len(set(text))}")
print(f"\n📖 First 500 characters:")
print(text[:500])

In [ ]:
# Step 2: Preprocess the text

# Create character to index mapping
chars = sorted(list(set(text)))
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

vocab_size = len(chars)
print(f"📝 Vocabulary size: {vocab_size}")
print(f"\n📋 Character mapping (first 20):")
for i, ch in enumerate(chars[:20]):
    print(f"  '{ch}' → {i}", end="  ")
    if (i + 1) % 5 == 0:
        print()

In [ ]:
# Step 3: Create training sequences

# We'll use a smaller subset for faster training in this demo
text_subset = text[:100000]  # Use first 100K characters

# Sequence length (how many characters to look at)
seq_length = 100

# Create input-output pairs
sequences = []
next_chars = []

for i in range(0, len(text_subset) - seq_length, 3):  # Step by 3 to create more diverse samples
    sequences.append(text_subset[i:i + seq_length])
    next_chars.append(text_subset[i + seq_length])

print(f"🔢 Number of training sequences: {len(sequences):,}")

# Show an example
print(f"\n📌 Example sequence:")
print(f"Input:  '{sequences[0]}'")
print(f"Target: '{next_chars[0]}'")

In [ ]:
# Step 4: One-hot encode the sequences

print("⏳ Creating one-hot encoded arrays...")

# Create numpy arrays
X = np.zeros((len(sequences), seq_length, vocab_size), dtype=np.float32)
y = np.zeros((len(sequences), vocab_size), dtype=np.float32)

for i, (seq, target) in enumerate(zip(sequences, next_chars)):
    for t, char in enumerate(seq):
        X[i, t, char_to_idx[char]] = 1
    y[i, char_to_idx[target]] = 1

print(f"✅ Data shapes:")
print(f"   X (input):  {X.shape}  - (samples, timesteps, features)")
print(f"   y (target): {y.shape}  - (samples, vocab_size)")

In [ ]:
# Step 5: Build the LSTM model for text generation

def build_text_generator(seq_length, vocab_size):
    model = keras.Sequential([
        layers.LSTM(256, input_shape=(seq_length, vocab_size), return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(256),
        layers.Dropout(0.2),
        layers.Dense(vocab_size, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

model = build_text_generator(seq_length, vocab_size)
model.summary()

In [ ]:
# Step 6: Train the model

print("🚀 Starting training...")
print("(This may take a few minutes. For better results, increase epochs!)\n")

# Train for a few epochs (increase for better results)
history = model.fit(
    X, y,
    batch_size=128,
    epochs=5,  # Increase this for better results (10-20 recommended)
    validation_split=0.1,
    verbose=1
)

In [ ]:
# Step 7: Visualize training progress

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Plot Loss
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Model Loss Over Time')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot Accuracy
axes[1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Model Accuracy Over Time')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Step 8: Generate text!

def sample_with_temperature(predictions, temperature=1.0):
    """
    Sample from predictions with temperature.
    Higher temperature = more random, lower = more deterministic.
    """
    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions + 1e-10) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

def generate_text(model, seed_text, length=200, temperature=0.5):
    """
    Generate text starting from a seed text.
    """
    generated = seed_text
    
    for _ in range(length):
        # Prepare input
        x_pred = np.zeros((1, seq_length, vocab_size))
        for t, char in enumerate(generated[-seq_length:]):
            if char in char_to_idx:
                x_pred[0, t, char_to_idx[char]] = 1
        
        # Predict next character
        predictions = model.predict(x_pred, verbose=0)[0]
        next_idx = sample_with_temperature(predictions, temperature)
        next_char = idx_to_char[next_idx]
        
        generated += next_char
    
    return generated

# Generate text with different temperatures
seed_text = text[:seq_length]  # Use beginning of text as seed

print("="*80)
print("🎭 GENERATED TEXT EXAMPLES")
print("="*80)

for temp in [0.2, 0.5, 1.0]:
    print(f"\n📝 Temperature = {temp} (higher = more creative)")
    print("-"*60)
    generated = generate_text(model, seed_text, length=200, temperature=temp)
    print(generated[seq_length:])  # Print only the generated part
    print()

---
## 7. 🧪 Experiment Yourself!

Try modifying the following and observe the results:

1. **Temperature**: Change temperature values in text generation
2. **Epochs**: Train for more epochs (10-20) for better results
3. **LSTM units**: Try different numbers (128, 512)
4. **Sequence length**: Try 50 or 200 instead of 100
5. **Replace LSTM with GRU**: Compare training speed and quality

In [ ]:
# YOUR EXPERIMENTS HERE

# Try generating with your own seed text!
my_seed = "To be or not to be, that is the question whether tis nobler in the mind to suffer the slings and"

# Make sure seed is exactly seq_length characters
if len(my_seed) < seq_length:
    my_seed = my_seed + " " * (seq_length - len(my_seed))
else:
    my_seed = my_seed[:seq_length]

print("🎭 Your custom generation:")
print("-"*60)
custom_generated = generate_text(model, my_seed, length=300, temperature=0.6)
print(custom_generated)

---
## 8. Comparing RNN, LSTM, and GRU

Let's build and compare all three architectures!

In [ ]:
# Build comparable models with RNN, LSTM, and GRU

def build_rnn_model(units=64):
    return keras.Sequential([
        layers.SimpleRNN(units, input_shape=(seq_length, vocab_size), return_sequences=True),
        layers.SimpleRNN(units),
        layers.Dense(vocab_size, activation='softmax')
    ])

def build_lstm_model(units=64):
    return keras.Sequential([
        layers.LSTM(units, input_shape=(seq_length, vocab_size), return_sequences=True),
        layers.LSTM(units),
        layers.Dense(vocab_size, activation='softmax')
    ])

def build_gru_model(units=64):
    return keras.Sequential([
        layers.GRU(units, input_shape=(seq_length, vocab_size), return_sequences=True),
        layers.GRU(units),
        layers.Dense(vocab_size, activation='softmax')
    ])

# Create models
rnn_model = build_rnn_model()
lstm_model = build_lstm_model()
gru_model = build_gru_model()

# Compare parameter counts
print("📊 Model Comparison:")
print("="*50)
print(f"{'Model':<15} {'Parameters':>15} {'Relative Size':>15}")
print("-"*50)

models = [
    ('Simple RNN', rnn_model),
    ('LSTM', lstm_model),
    ('GRU', gru_model)
]

base_params = rnn_model.count_params()
for name, m in models:
    params = m.count_params()
    relative = params / base_params
    print(f"{name:<15} {params:>15,} {relative:>14.1f}x")

---
## 9. Summary & Key Takeaways

### What We Learned:

| Concept | Description |
|---------|-------------|
| **RNNs** | Neural networks with memory for sequence processing |
| **Vanishing Gradient** | Why simple RNNs fail on long sequences |
| **LSTM** | Solves vanishing gradient with gates (forget, input, output) |
| **GRU** | Simplified LSTM with fewer parameters |
| **Text Generation** | Character-by-character prediction using RNNs |
| **Temperature** | Controls randomness in text generation |

### When to Use What:

- **Simple RNN**: Short sequences, simple patterns
- **LSTM**: Long sequences, complex dependencies
- **GRU**: Similar tasks as LSTM but faster training needed

### Next Steps:

1. Try training on different datasets (song lyrics, code, etc.)
2. Explore Bidirectional LSTMs
3. Learn about Attention mechanisms (Week 3)
4. Study Transformers - the next evolution! (Week 3-4)

---
## 10. 📝 Practice Exercises

### Exercise 1: Temperature Exploration
Generate text with temperatures: 0.1, 0.3, 0.7, 1.5. What patterns do you observe?

### Exercise 2: GRU vs LSTM
Replace the LSTM model with GRU. Compare:
- Training time
- Final loss
- Output quality

### Exercise 3: Bidirectional LSTM
Modify the model to use `layers.Bidirectional(layers.LSTM(...))`. How does it affect performance?

### Exercise 4: Custom Dataset
Train the model on a different text dataset (your favorite book, song lyrics, etc.)

In [ ]:
# Space for your exercise solutions!

# Exercise 1: Temperature exploration
# TODO: Generate text with different temperatures and compare


# Exercise 2: GRU comparison
# TODO: Build and train a GRU model


# Exercise 3: Bidirectional LSTM
# TODO: Create a bidirectional model


---
## 11. 📚 Additional Resources

### Reading:
- [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) - Chris Olah's classic blog post
- [The Unreasonable Effectiveness of RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) - Andrej Karpathy

### Video:
- [RNN & LSTM - Stanford CS231n](https://www.youtube.com/watch?v=6niqTuYFZLQ)

### Code:
- [TensorFlow RNN Tutorial](https://www.tensorflow.org/text/tutorials/text_generation)
- [PyTorch Sequence Models](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)

---

**🎉 Congratulations on completing the RNN notebook!**

*Next week: Variational Autoencoders (VAEs)*